# Setup Libraries & Constant

In [33]:
import sys
import ast
import pandas as pd
import docplex.mp
from collections import namedtuple
from docplex.mp.model import Model
from docplex.mp.environment import Environment
import random

# Define Constant

## Country & Economics

In [34]:
int_cod = ('JPN',  'LKA',  'HKG',  'CHN',  'SGP',  'MYS',  'KOR',  'IND',  'OAS',  'PAK',  'IDN',  'PHL',  'BGD',  'THA',  'VNM',  'KHM',  'MMR')
int_country = {'BD': 'BGD','CN': 'CHN','HK': 'HKG','IN': 'IND','ID': 'IDN', 'JP': 'JPN',
  'KH': 'KHM','KR': 'KOR','MM': 'MMR', 'MY': 'MYS','PK': 'PAK','PH': 'PHL','SG': 'SGP',
   'LK': 'LKA',  'TW': 'OAS',  'TH': 'THA',  'VN': 'VNM' }
con_int = dict(zip(int_country.values(),int_country.keys()))

assert len(int_cod),len(int_country)
assert len([i for i in int_country.values() if i not in int_cod]) ==0
regions_cod = ('AFR','EUR','ELT','WLT','MDE','NAM','OCE')
economics = tuple(int_cod + regions_cod)

## Vessel

In [35]:
VESSEL_TYPE = [
    ('Feeder',      3000,   8,   170, 1700, 146 ),
    ('Panamax',     8500,   11,  275, 1100, 220),
    ('Neo_Panamax', 14000,  13,  335, 600,  326),
    ('ULCV',        25000,  14,  390, 200,  506)
]
# 170 275 335 390
Vessel_type = namedtuple('vessel_type',['type','teu','draft','length','quantity','deploy_cost'])
# Cost in week base (k$)
vtypes = [Vessel_type(*v) for v in VESSEL_TYPE]
vtypes

# Note. Don't forget to adjust ship building year
# Vessel 				Size			DWT(Med)	Depth		Length (Min,Med,Max,Mean)
# Feeder 				(<3000 Teu)		20k			<=8			78,170,247,168
# Panamax  			(3000-8500 Teu)		65k			<=11		175,275,366,277
# Post / Neo Panamax 	(8500-14000 Teu)120k		<=13		298,336,400,338
# ULCV 				(>14000 Teu)		190k		<=14		353,399,400,389

[vessel_type(type='Feeder', teu=3000, draft=8, length=170, quantity=1700, deploy_cost=146),
 vessel_type(type='Panamax', teu=8500, draft=11, length=275, quantity=1100, deploy_cost=220),
 vessel_type(type='Neo_Panamax', teu=14000, draft=13, length=335, quantity=600, deploy_cost=326),
 vessel_type(type='ULCV', teu=25000, draft=14, length=390, quantity=200, deploy_cost=506)]

In [36]:
vtypes[3].teu


25000

# Import data

## Demand

In [37]:
dem_matrix = pd.read_csv('load_matrix_2015.csv').iloc[:,1:]
# Demand = namedtuple('Frm',['teu','draft','length','quantity','deploy_cost'])
# OD = Origin Destination
demands = {}
for f in economics:
    for t in economics:
        if f==t : continue
        demands[f,t] = dem_matrix[(dem_matrix.From==f)&(dem_matrix.To==t)].TEU.sum()
assert len(demands) == len(economics) *(len(economics)-1)
print(f"Total Demand {len(demands)}")    #552
"('JPN', 'LKA') : ",demands[('JPN', 'LKA')]

Total Demand 552


("('JPN', 'LKA') : ", 9036)

## Port

In [38]:
master_port = pd.read_csv('ports_master.csv').iloc[:,1:]
# port = master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r']].itertuples(index=False)
PORTS = list(master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu']].itertuples(index=False, name=None))
# Port = namedtuple('port',['LOCODE','draft0','draft1','draft2','draft3','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu'])
# PORT[]
PORTS [55:61],len(PORTS)


([('PHMNL', 850, 1100, 1675, 0, 8375, 20625, 38125, 38125, 75, 1),
  ('PKBQM', 0, 0, 670, 780, 24831, 65081, 122581, 122581, 51, 30),
  ('PKKHI', 0, 0, 1675, 1560, 46012, 131762, 254262, 254262, 128, 0),
  ('SGSIN', 1700, 1375, 0, 17160, 4018, 5768, 8268, 8268, 65, 42),
  ('THSGZ', 510, 0, 0, 0, 5399, 15899, 30899, 30899, 47, 1),
  ('THBKK', 2890, 0, 0, 0, 5399, 15899, 30899, 30899, 47, 1)],
 68)

In [39]:
# EX_PORT = , 0, 0, 0, 0, 23110, 35360, 52860, 52860, 0, 31)
for reg in regions_cod:
    sc = (reg, 10000, 10000, 10000, 50000, 0, 0, 0, 0, 0, 999)
    PORTS.append(sc)
# len(sc),len(ports[2]),sc
len(PORTS) #75
# High transhipment, Low THC, due ???, unlimited berth

75

In [40]:
interest_port = tuple(master_port.LOCODE.tolist())
total_port = tuple(master_port.LOCODE.tolist() + list(regions_cod))
total_port[:3],total_port[-3:] 

(('BDCGP', 'CNDCB', 'CNYTG'), ('MDE', 'NAM', 'OCE'))

In [41]:
Ports_info = namedtuple('ports_info',['lOCODE','draft0_r','draft1_r','draft2_r','draft3_r','due_0', 'due_1', 'due_2', 'due_3', 'thc','tranship'])
ports = [Ports_info(*p) for p in PORTS]
ports [:2],len(ports)

([ports_info(lOCODE='BDCGP', draft0_r=3060, draft1_r=0, draft2_r=0, draft3_r=0, due_0=36380, due_1=104630, due_2=202130, due_3=202130, thc=52, tranship=1),
  ports_info(lOCODE='CNDCB', draft0_r=0, draft1_r=0, draft2_r=0, draft3_r=2340, due_0=15797, due_1=24547, due_2=37047, due_3=37047, thc=89, tranship=31)],
 75)

In [42]:
ports[0].due_0 == ports[0][5]
ports[9][8]
ports[0][-2]

52

In [43]:
port_country = [(int_country[p[:2]],i,p) for i,p in enumerate(total_port) if len(p)==5]
port_country += [(p,i,p) for i,p in enumerate(total_port) if len(p)==3]
port_country[:7],port_country[-7:]

([('BGD', 0, 'BDCGP'),
  ('CHN', 1, 'CNDCB'),
  ('CHN', 2, 'CNYTG'),
  ('CHN', 3, 'CNQZH'),
  ('CHN', 4, 'CNFOC'),
  ('CHN', 5, 'CNLYG'),
  ('CHN', 6, 'CNDLC')],
 [('AFR', 68, 'AFR'),
  ('EUR', 69, 'EUR'),
  ('ELT', 70, 'ELT'),
  ('WLT', 71, 'WLT'),
  ('MDE', 72, 'MDE'),
  ('NAM', 73, 'NAM'),
  ('OCE', 74, 'OCE')])

## Port Call

In [44]:
port_call_df = pd.read_csv('Port_call_master.csv').iloc[:,1:]
port_call_df = port_call_df[port_call_df.Calls_Count>0].reset_index(drop=True)
def str2list(x:str):
    return list(ast.literal_eval(x))
port_call_df.LOCODE_Calls = port_call_df.LOCODE_Calls.apply(str2list)
port_call_df.Calls = port_call_df.Calls.apply(str2list)
port_call_df[['Calls','PC_Count','Area_Calls','Area_Cover','Turn_Around','Week','Calls_Count']].iloc[102]

Calls          [CNXMG, TWKHH, TWTXG, CNXMG, TWKHH, CNXMG]
PC_Count                                                6
Area_Calls                                            INT
Area_Cover                                              3
Turn_Around                                             7
Week                                                    1
Calls_Count                                             6
Name: 102, dtype: object

In [45]:
PORTCALL = list(port_call_df[['Calls','PC_Count','Area_Calls','Area_Cover','Turn_Around','Week','Calls_Count']].itertuples(index=False, name=None))
PORTCALL[92]
port_call_df[['LOCODE_Calls','Calls','PC_Count','Area_Calls','Area_Cover','Turn_Around','Week','Calls_Count']].iloc[92]
# port_call_df.Calls_Count.value_counts()

LOCODE_Calls    [CNNBG, CNSHG, KRPUS, USLGB, KRPUS, CNNBG]
Calls               [NAM, KRPUS, CNNBG, CNSHG, KRPUS, NAM]
PC_Count                                                 6
Area_Calls                                         INT,NAM
Area_Cover                                               7
Turn_Around                                             42
Week                                                     6
Calls_Count                                              6
Name: 92, dtype: object

In [46]:
# Store as vector [0,0,0,]
routes_vec = []
cnt,ind = 0,0
conv_bin = lambda x: 0 if x==0 else 1
for ins in PORTCALL:
    pc = ins[0]
    route = []
    legs = ins[-1]
    cnt += legs
    wk = ins[-2]
    for l in range(legs):
        call_port = pc[l]
        # print(call_port)
        call = [conv_bin(call_port == p) for p in total_port]
        route.append(tuple(call))
    
    routes_vec.append((wk,legs,tuple(pc),tuple(route)))
    ind += 1
len(routes_vec),cnt,routes_vec[3][:3]
# [ind,size,leg1 * [0,0,0,],leg2,....]

(431, 3501, (1, 3, ('SGSIN', 'KHKOS', 'SGSIN')))

In [62]:
# Store as index 
routes_ind = []
cnt,ind = 0,0
conv_bin = lambda x: 0 if x==0 else 1
for ins in PORTCALL:
    pc = ins[0]
    route = []
    legs= ins[-1]
    cnt += legs
    wk = ins[-2]
    for l in range(legs):
        call_port = pc[l]
        # print(call_port)
        try:
            call = total_port.index(call_port)
        except:
            print(pc,len(routes_ind),call_port)
            raise
        route.append(call)
    routes_ind.append((wk,legs,tuple(pc),tuple(route)))
    ind += 1

len(routes_ind),cnt,routes_ind[3][:4]
# [size,legs,pc(locode),leg2,....]

(431, 3501, (1, 3, ('SGSIN', 'KHKOS', 'SGSIN'), (58, 39, 58)))

## Iterator

In [48]:
# port_capacity = 
_interest_port  = len(interest_port)
_total_port     = len(total_port)
_ports          = len(ports)
_port_country   = len(port_country)

_interest_port,_total_port,_ports,_port_country,type(interest_port),type(total_port),type(ports),type(port_country)

(68, 75, 75, 75, tuple, tuple, list, list)

In [49]:
# econ demand
_demands        = len(demands)
_int_cod        = len(int_cod)# 3 Digit
_int_country    = len(int_country)  # 2 Digit
_regions_cod    = len(regions_cod)
_economics      = len(economics)

_demands,_int_cod,_int_country,_regions_cod,_economics,type(demands),type(int_cod),type(int_country),type(regions_cod),type(economics)

(552, 17, 17, 7, 24, dict, tuple, dict, tuple, tuple)

In [50]:
# route preparing
routes_ind
routes_vec
_routes = len(routes_ind)

# Vessel 
_vtypes = len(vtypes)
_routes ,_vtypes,type(routes_ind),type(vtypes)

(431, 4, list, list)

In [66]:
port_country

[('BGD', 0, 'BDCGP'),
 ('CHN', 1, 'CNDCB'),
 ('CHN', 2, 'CNYTG'),
 ('CHN', 3, 'CNQZH'),
 ('CHN', 4, 'CNFOC'),
 ('CHN', 5, 'CNLYG'),
 ('CHN', 6, 'CNDLC'),
 ('CHN', 7, 'CNTXG'),
 ('CHN', 8, 'CNNSA'),
 ('CHN', 9, 'CNXMG'),
 ('CHN', 10, 'CNSHK'),
 ('CHN', 11, 'CNQDG'),
 ('CHN', 12, 'CNYTN'),
 ('CHN', 13, 'CNNBG'),
 ('CHN', 14, 'CNSHG'),
 ('HKG', 15, 'HKHKG'),
 ('IDN', 16, 'IDSRG'),
 ('IDN', 17, 'IDSUB'),
 ('IDN', 18, 'IDJKT'),
 ('IND', 19, 'INTUT'),
 ('IND', 20, 'INCCU'),
 ('IND', 21, 'INDMQ'),
 ('IND', 22, 'INCOK'),
 ('IND', 23, 'INVTZ'),
 ('IND', 24, 'INMRM'),
 ('IND', 25, 'INHZA'),
 ('IND', 26, 'INMAA'),
 ('IND', 27, 'INPAV'),
 ('IND', 28, 'INMUN'),
 ('IND', 29, 'INNSA'),
 ('JPN', 30, 'JPMOJ'),
 ('JPN', 31, 'JPHKT'),
 ('JPN', 32, 'JPYKK'),
 ('JPN', 33, 'JPSMZ'),
 ('JPN', 34, 'JPOSA'),
 ('JPN', 35, 'JPNGO'),
 ('JPN', 36, 'JPUKB'),
 ('JPN', 37, 'JPTYO'),
 ('JPN', 38, 'JPYOK'),
 ('KHM', 39, 'KHKOS'),
 ('KOR', 40, 'KRINC'),
 ('KOR', 41, 'KRUSN'),
 ('KOR', 42, 'KRKAN'),
 ('KOR', 43, 'KRPUS')

In [51]:
rv =    [(r,v)        for r in range(_routes) for v in range(_vtypes)] 
rol =   [(r, o, l)    for r in range(_routes) for o in range(_economics) for l in range(routes_ind[r][1])]
rolp =  [(r,o,l,routes_ind[r][3][l])    for r in range(_routes) for o in range(_economics) for l in range(routes_ind[r][1])]
ope =   [(o,_p,economics.index(e))     for e,_p,p in port_country for o in range(_economics)]   
pv =    [(p,v)        for p in range(_total_port) for v in range(_vtypes)]
po =    [(p,o)        for p in range(_total_port) for o in range(_economics)]  
#routes_ind[2]
# po[32]
len(rolp),len(rol),len(po),len(ope)

(84024, 84024, 1800, 1800)

# Setup Model

In [52]:
env = Environment()
env.print_information()
mdl = Model("Container_network")

* system is: Windows 64bit
* Python version 3.8.5, located at: C:\ProgramData\Anaconda3\python.exe
* docplex is present, version is 2.18.200
* CPLEX library is present, version is 20.1.0.0, located at: C:\ProgramData\Anaconda3\lib\site-packages
* pandas is present, version is 1.1.3


## Decision Variable

In [53]:

# Vessel in route (Ves=>)
ves_deploy = mdl.integer_var_dict(rv, ub=5, name="ves_deploy") # Maximum 7 day/weeks call

# # Load, Unload, and Carried
load = mdl.integer_var_dict(rolp,name="load")
unload = mdl.integer_var_dict(rolp, name="unload")
carried = mdl.integer_var_dict(rolp, name="carried")

            # # print(v,o,r[0],r[])
# Import,Export
imp = mdl.integer_var_dict(ope,name="imp")
exp = mdl.integer_var_dict(ope,name="exp")
trade = mdl.integer_var_dict(ope,name="trade")

In [61]:
ope[0],rolp[3] #

((0, 0, 12), (0, 1, 0, 44))

## Cal Var


In [55]:
route_cap =  mdl.integer_var_list(range(_routes),name="route_cap")
port_thrp =  mdl.integer_var_list(range(_total_port),name="port_thrp")
# port_load =  mdl.integer_var_dict(po,name="port_load")
# port_unload = mdl.integer_var_dict(po,name="port_unload")
# port_trans  = mdl.integer_var_dict(po,name="trans") 
port_load =  mdl.integer_var_dict(ope,name="port_load")
port_unload = mdl.integer_var_dict(ope,name="port_unload")
port_trans  = mdl.integer_var_dict(ope,name="trans") 
port_calls =  mdl.integer_var_dict(pv,name="port_calls")

len(port_thrp )

# mdl.add_constraint(
#                 mdl.sum((price_prod_econ[f,p] * Trade_demand[f,t,p]* cont_conv[p] ) for p in products) 
#                 == load_from_to[f,t]
#             ) 

75

## Constraint

### C1 Capacity of each route

In [56]:
for r in range(_routes):
    mdl.add_constraint(
            ## deploy * size 
            mdl.sum((ves_deploy[(r,v)]*vtypes[v].teu) for v in range(_vtypes)) 
            == route_cap[r],f"Cap route {r}"
        ) 

In [ ]:
r

### C2 Vessel Availability (Vessel deploy according to weekly service)
### C3 Port Total Call (by types)

In [57]:
for v in range(_vtypes):
    mdl.add_constraint(
            ## deploy * size 
            mdl.sum((ves_deploy[(r,v)]*routes_ind[r][0]) for r in range(_routes)) 
            <= vtypes[v].quantity,f"Availability type {v}"
        )
    for _p,p in enumerate(total_port): 
        mdl.add_constraint( mdl.sum(ves_deploy[(r,v)] for r in range(_routes) 
            for l in range(1,routes_ind[r][1])  if routes_ind[r][2][l]==p ) # Use 1 to reduce end duplicate 
            ==port_calls[(_p,v)]),f"Total Call port {p}, type {v}"

### C4 Carried less than vessel capacity
### C5 Port Flow (vessel) Continuity (per each berthing) 
### C6 Circular Route (PC[0] ==PC[-1])

In [58]:
for _rol in rolp :
    r,o,l,p = _rol
    mdl.add_constraint( 
        ## z < route_cap
        carried[_rol] <= route_cap[r],f"Cap & Carried  {r},{o},{l}"
    )
    if l==routes_ind[r][1]-1:     # Skip last portcall since is identical with first portcall see C5
        p0 = routes_ind[r][2]
        mdl.add_constraint(carried[(r,o,0,p0)] == carried[_rol],f"Circ Carried  {r},{o},{l},{p}")
        mdl.add_constraint(load[(r,o,0,p0)] == load[_rol]      ,f"Circ Load  {r},{o},{l},{p}")
        mdl.add_constraint(unload[(r,o,0,p0)] == unload[_rol]  ,f"Circ unload  {r},{o},{l},{p}")
        continue
    _p = routes_ind[r][3][l+1]
    mdl.add_constraint(carried[(r,o,l+1,_p)] == carried[_rol] + load[_rol] - unload[_rol],f"R Continuity {r},{o},{l}")

    

### C7 Port Load, Unload, No internal trade, & Throughput
TH = load + unload
For port in e can only load cargoes from e

In [27]:

for _o,_p,_e in ope:
        # Port load
    mdl.add_constraint( mdl.sum(load[(r,o,l,p)] 
                            for r,o,l,p in rolp if p==_p and o==_o and l!=0) 
                        == port_load[(_o,_p,_e)],
                        f"Port Load  {_p},{_o}")

    # Port unload
    mdl.add_constraint( mdl.sum(unload[(r,_o,l,p)]  
                            for r,o,l,p in rolp if p==_p and o==_o and l!=0) 
                        == port_unload[(_o,_p,_e)],
                        f"Port Unload {_p},{_o},{_e}")
    # if _o == _e:
    #     # Port unload
    #     mdl.add_constraint( port_unload[(_p,_o)] == 0,
    #                             f"Port Unload Fix=0 {_p},{_o}")


for _p,p in enumerate(total_port):
# Port Throughput
    mdl.add_constraint( mdl.sum(load[(r,o,l,p_)] for r,o,l,p_ in rolp if p_==_p and l!=0)
                        + mdl.sum(unload[(r,o,l,p_)] for r,o,l,p_ in rolp if p_==_p and l!=0)
                        ==port_thrp[_p],f"Port Throughput {p}")

    

In [60]:

_p = 0
[(r,o,l,p) for r,o,l,p in rolp if p==_p and o==_o and l!=0]

[(9, 23, 1, 0),
 (16, 23, 1, 0),
 (33, 23, 2, 0),
 (35, 23, 1, 0),
 (37, 23, 2, 0),
 (42, 23, 1, 0),
 (47, 23, 1, 0),
 (151, 23, 2, 0),
 (373, 23, 9, 0)]

### C8 Berth Draft Limitation

In [29]:
for v in range(_vtypes):
    _v = _vtypes - v -1
    for _p,p in enumerate(total_port) :
        mdl.add_constraint( mdl.sum(port_calls[(_p,v_)] for v_ in range(_v,_vtypes))
                        == mdl.sum((port_calls[(_p,v_)] * vtypes[v_].length) for v_ in range(_v,_vtypes))
                        ,f"Berth Limitation port {p} type {v}")



### C9 Trade for each country
### C10 Demand Satisfied (Only in interested Area)
### C11 No internal trade

In [30]:
for _f,f in enumerate(economics):           # @ Economy f
    fports = [(_p,p) for c,_p,p in port_country if c == f]
    for _t,t in enumerate(economics):       # Partner
        if (f in regions_cod and t in regions_cod) :
            # External frame relaxed 
            continue
        if (f==t):  # Export Case
            mdl.add_constraints([trade[(_t,_p,_f)] == port_load[(_t,_p,_f)] for _p,p in fports])
            
            # if  f in regions_cod: # External Case  No tranship
            #     mdl.add_constraints([trade[(_t,_p,_f)] == port_unload[(_t,_p,_f)] for _p,p in fports],
            #                  [f"EXT Imp = unload for {p}" for _p,p in fports])
        else:       # Import Case
            # Import = Unload - load
            mdl.add_constraints(trade[(_t,_p,_f)] == 
                                port_unload[(_t,_p,_f)]- port_load[(_t,_p,_f)] for _p,p in fports )
            # Sum import = demand
            mdl.add_constraint(mdl.sum(trade[(_t,_p,_f)] for _p,p in fports)
                            == demands[(t,f)],
                            f"Demand  from {t} satisfy for {f}")

        # Tran = Unload + load - import
        mdl.add_constraints(port_trans [(_t,_p,_f)] == port_unload[(_t,_p,_f)] + port_load[(_t,_p,_f)]
                            - trade[(_t,_p,_f)] for _p,p in fports )

        # ## Sum every port in f (demand satisfied)
        # mdl.add_constraint(mdl.sum(imp[(o,_p,e)] for o,_p,e in ope if o==_t and e == f)
        #                     == demands[(t,f)],
        #                     f"Demand (in) satisfy for {f},{t}")
        
            # else:
            # Sum every port in f (demand relaxed satisfied)
            #     mdl.add_constraint(mdl.sum(trade[(_t,_p,_f)] for o,_p,e in ope if o==_t and e == f)
            #                     <= demands[(t,f)],
            #                     f"Demand (in) satisfy for {f},{t}")

            

## Cost Calculation

In [31]:
# Vessel deploy cost
# ** Assign cost for external 
ves_deploy_cost = mdl.sum(ves_deploy[(r,v)] * vtypes[v].deploy_cost for r,v in rv)

# Load & unload cost
# ** Adjust cost for external 
handling_cost = mdl.sum(port_thrp[_p] * ports[_p][-2] for _p in range(_total_port))

# Transhipment cost
handling_cost = mdl.sum(port_trans [(o,_p,e)] * ports[_p][-1] for o,_p,e in ope)

# Port Call cost

In [32]:
mdl.minimize(handling_cost + ves_deploy_cost +handling_cost)
mdl.print_information()
# mdl.parameters.mip.tolerances.mipgap = 10

Model: Container_network
 - number of variables: 265402
   - binary=0, integer=265402, continuous=0
 - number of constraints: 197458
   - linear=197458
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [33]:
s = mdl.solve(log_output=True)
if s is not None:
    mdl.report()
else:
    print("* model is infeasible")

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Infeasibility row 'Port Unload 0,0,12':  0  = 22685.
Presolve time = 0.42 sec. (463.17 ticks)

Root node processing (before b&c):
  Real time             =    0.45 sec. (490.20 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.45 sec. (490.20 ticks)
* model is infeasible


In [375]:
mdl.print_information()

In [313]:
for v in range(_vtypes) :
    for r,o,l,p in rolp :
        try:
            if l !=0:
                x = ves_deploy[(r,v)] * ports[p][5+v]
        except:
            print(v,r,o,l,p)
            raise

0 7 0 2 74


IndexError: list index out of range

In [314]:
# mdl.add_constraint(
#                 mdl.sum((price_prod_econ[f,p] * Trade_demand[f,t,p]* cont_conv[p] ) for p in products) 
#                 == load_from_to[f,t]
#             ) 
routes_ind[7]

(2, 3, ('OCE', 'SGSIN', 'OCE'), (74, 58, 74))

### CX Port Cap ?

In [141]:
port_country
ope = [(p[:2],p,e)for p in interest_port for e in economics]
len(ope),_ports*_economics
        


(1632, 1632)

## Objective & KPI

In [4]:
# Load from port to route
cnt = 0
for r in range(_routes):
    for o in range(_economics):
        for l in range(routes_ind[r][1]):
            cnt +=1
# Unload from route to port
unload = mdl.integer_var_matrix( keys1=matrix_region,keys2=products, ub=0.8, name="price_prod_econ")
# Carried to next leg
carried = mdl.integer_var_matrix( keys1=matrix_region,keys2=products, ub=0.8, name="price_prod_econ")

x = m.integer_var_dict((f, r, r['Leg'], d)
                       for f in economics
                       for r in routes
                       for d in r['Call'])

                       


NameError: name 'mdl' is not defined

In [56]:
len(economics),len(routes),3522/435

(24, 435, 8.096551724137932)

In [53]:
vtypes = (0,1,2,3)
cnt = 0
sel = int(random.random() * len(routes) //1)    # 3522 total leg
print(sel)
for r in routes:
    for o in economics:
        for l in range(r[1]):
            # print(v,o,r[0],r[])
            cnt +=1
cnt
# 84528 = 3522 * 

366


84528

In [5]:
routes = [{'Name':'A','Leg':5,'Call':['X','Y','Z']},{'Name':'B','Leg':7,'Call':['X','Y','X']},{'Name':'A','Leg':9,'Call':['Z','Y','Z']}]
# [(i,j,p) for i in routes for j in i for p in i['Call']]

In [ ]:
# Internal Teu Allocation (Economics to port
for c in matrix_region for e in matrix_region for 

In [ ]:
# https://stackoverflow.com/questions/55821484/how-to-set-a-four-dimension-variable-in-docplex-with-python 
x = m.binary_var_dict((i, l, t, v, r)
                       for i in types
                       for l in locations
                       for t in times
                       for v in vehicles
                       for r in routes)

for i in types:
    for l in locations:
       for t in times:
          for v in vehicles:
             for r in routes:
                print x[i, l, t, v, r]

x = [[[[[m.binary_var('%d_%d_%d_%d_%d' % (i, l, t, v, r))
        for r in routes]
        for v in vehicles]
    for t in times]
    for l in locations]
    for i in types]

for i in types:
for l in locations:
    for t in times:
        for v in vehicles:
            for r in routes:
            print x[i][l][t][v][r]

In [ ]:
from docplex.mp.model import Model

# Data

r=range(1,3)

i=[(a,b,c,d) for a in r for b in r for c in r for d in r]

print(i)

mdl = Model(name='model')

#decision variables
mdl.x=mdl.integer_var_dict(i,name="x")

# Constraint
for it in i:
    mdl.add_constraint(mdl.x[it] == it[0]+it[1]+it[2]+it[3], 'ct')

mdl.solve()

# Dislay solution
for it in i:
    print(" x ",it," --> ",mdl.x[it].solution_value); 

# Model

In [144]:
docplex.mp.context.Context.read_settings()

TypeError: read_settings() missing 1 required positional argument: 'self'

In [ ]:
def demand_model1(Trade_demand,R_TEU,cont_conv = containerized):
    products = tuple([c for c in cont_conv.keys() if cont_conv[c]!=0 ])
    global int_cod,matrix_region
    mdl = Model(name='trade_to_teu_1')
    
    price_prod_econ = mdl.continuous_var_matrix( keys1=matrix_region,keys2=products, ub=0.8, name="price_prod_econ") 
    # (TEU/k$)  : Less Value => Expensive Cargo
    load_from_to =  mdl.continuous_var_matrix(keys1=matrix_region, keys2=matrix_region, name="load_from_to")
    load_total =  mdl.continuous_var_dict(keys=matrix_region, lb=10, ub = R_TEU, name="load_total")

    for f in matrix_region:
        for t in matrix_region:
            if f==t :
                continue
            mdl.add_constraint(
                mdl.sum((price_prod_econ[f,p] * Trade_demand[f,t,p]* cont_conv[p] ) for p in products) 
                == load_from_to[f,t]
            ) 
    for f in matrix_region:
        mdl.add_constraint(
            mdl.sum(load_from_to[f,t] for t in matrix_region if t!=f ) + mdl.sum(load_from_to[k,f] for k in matrix_region if k!=f)
            == load_total[f] 
            ) 
    
    total_err = mdl.sum((R_TEU[f] - load_total[f])**2 for f in int_cod)
    mdl.minimize(total_err)
    
    sol = mdl.solve()
    if sol is not None:
        # mdl.print_information()
        mdl.report()
        price = mdl.solution.get_value_dict(price_prod_econ)
        price_df = pd.DataFrame([{'Econ':k[0],'Product':k[1],'Price':v} for k,v in price.items()],columns = ['Econ','Product','Price'])
        mid = price_df.groupby('Product').Price.describe()[['50%', 'mean','min','max']]
        mid['Min_mid'] = mid[['50%', 'mean']].min(axis=1)
        mid['Max_mid'] = mid[['50%', 'mean']].max(axis=1)
        mid['Max_Uratio'] = mid['max']/mid['Max_mid'] 
        mid['Max_Lratio'] = mid['min']/mid['Max_mid'] 
        obj = sol.get_objective_value()
        # print(f"solution for a cost of {mdl.objective_value}")
        # print_information(model)
        # print_solution(model)
        return mid,obj
    else:
        print("* model is infeasible")
        return None


def demand_model2(base_price,Trade_demand,R_TEU,cont_conv = containerized):
    products = tuple([c for c in cont_conv.keys() if cont_conv[c]!=0 ])
    global int_cod,matrix_region
    mdl = Model(name='trade_to_teu_2')

    load_from_to =  mdl.continuous_var_matrix(keys1=matrix_region, keys2=matrix_region, name="load_from_to")
    load_total =  mdl.continuous_var_dict(keys=matrix_region, lb=10,ub = R_TEU, name="load_total")
    range_prod_econ = mdl.continuous_var_matrix(keys1=matrix_region, keys2=products,lb=0.002,ub=5.0, name="range_prod_econ")

    for f in matrix_region:
        for t in matrix_region:
            if f==t :
                continue
            mdl.add_constraint(
                mdl.sum((base_price[p]* range_prod_econ[f,p] * Trade_demand[f,t,p]* cont_conv[p] ) for p in products) 
                == load_from_to[f,t]
            ) 
    for f in matrix_region:
        mdl.add_constraint(
            mdl.sum(load_from_to[f,t] for t in matrix_region if t!=f ) + mdl.sum(load_from_to[k,f] for k in matrix_region if k!=f)
            == load_total[f] 
            ) 
    
    total_err = mdl.sum((R_TEU[f] - load_total[f])**2 for f in int_cod)
    mdl.minimize(total_err)
    
    sol = mdl.solve()
    if sol is not None:
        # mdl.print_information()
        mdl.report()
        load_matrix = mdl.solution.get_value_dict(load_from_to)
        C_TEU = mdl.solution.get_value_dict(load_total)
        price_range = mdl.solution.get_value_dict(range_prod_econ)
        price_range_df = pd.DataFrame([{'Econ':k[0],'Product':k[1],'Price':v} for k,v in price_range.items()],columns = ['Econ','Product','Price'])
        # print(sol.get_objective_value())
        obj = sol.get_objective_value()
        # print(f"solution for a cost of {mdl.objective_value}")
        # print_information(model)
        # print_solution(model)
        return load_matrix,C_TEU,price_range_df,obj
    else:
        print("* model is infeasible")
        return None


# Misc

In [51]:
# master_port.to_csv('ports_master.csv')
# master_port = sel_port.merge(port_draft,on='LOCODE')
master_port = pd.read_csv('ports_master.csv').iloc[:,1:]
master_port

,LOCODE,Country,Flow_in,Max_TEU,Min_TEU,UNLOCODE,Port_Name,draft0,draft1,draft2,draft3,Sum_len,draft0_r,draft1_r,draft2_r,draft3_r,Sum_len_r
0,BDCGP,BD,1146693.0,2806,431,BDCGP,CHATTOGRAM,3124.0,0,0,0,3124.0,3060,0,0,0,3060
1,CNDCB,CN,208187.0,6931,2770,CNDCB,DA CHAN BAY,0.0,0,0,2430,2430.0,0,0,0,2340,2340
2,CNYTG,CN,217905.0,4253,542,CNYTG,YANTAI PT,237.0,0,1492,2876,4605.0,170,0,1340,2730,4240
3,CNQZH,CN,251114.0,6096,556,CNQZH,QINZHOU PT,0.0,0,0,1533,1533.0,0,0,0,1560,1560
4,CNFOC,CN,426504.0,16022,368,CNFOC,Fuzhou,300.0,1699,0,2034,4033.0,340,1650,0,1950,3940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,TWTPE,TW,1718336.0,20190,610,TWTPE,TAIPEI,0.0,0,0,1377,1377.0,0,0,0,1560,1560
64,TWKEL,TW,1870217.0,10933,450,TWKEL,KEELUNG (CHILUNG),3349.0,660,210,403,4622.0,3400,550,335,390,4675
65,TWKHH,TW,12887313.0,23964,180,TWKHH,KAOHSIUNG,1259.0,0,2878,2565,6702.0,1190,0,3015,2730,6935
66,VNHPH,VN,1963571.0,12726,357,VNHPH,HAIPHONG,7084.0,450,0,750,8284.0,7140,550,0,780,8470


In [50]:
master_port[master_port.Country =='MY']

,LOCODE,Country,Flow_in,Max_TEU,Min_TEU,UNLOCODE,Port_Name,draft0,draft1,draft2,draft3,Sum_len,draft0_r,draft1_r,draft2_r,draft3_r,Sum_len_r
46,MYBTU,MY,211721.0,2806,144,MYBTU,"BINTULU, SARAWAK",514.0,0,450,0,964.0,510,0,335,0,845
47,MYJHB,MY,642648.0,9404,216,MYJHB,JOHOR BAHRU,0.0,708,0,0,708.0,0,825,0,0,825
48,MYPEN,MY,606905.0,9404,653,MYPEN,PENANG (GEORGETOWN),1245.0,600,0,0,1845.0,1190,550,0,0,1740
49,MYPKG,MY,13626729.0,23112,144,MYPKG,PORT KLANG (PELABUHAN KLANG),94.0,1137,2388,1697,5316.0,170,1100,2345,1560,5175
50,MYTPP,MY,10256935.0,23656,1005,MYTPP,TANJUNG PELEPAS,0.0,0,0,5040,5040.0,0,0,0,5070,5070


In [39]:
# Round berth lengh
def berth_round(ber,ves):
    return ves*round(ber/ves)
#  170 275 335 390
master_port['draft0_r'] = master_port.draft0.apply(lambda x: berth_round(x,170))
master_port['draft1_r'] = master_port.draft1.apply(lambda x: berth_round(x,275))
master_port['draft2_r'] = master_port.draft2.apply(lambda x: berth_round(x,335))
master_port['draft3_r'] = master_port.draft3.apply(lambda x: berth_round(x,390))
# Cumulative (Large berth can accomodate smaller vessel)
# master_port['draft3_rc'] = master_port.draft3_r
# master_port['draft2_rc'] = master_port.draft2_r +  master_port.draft3_rc
# master_port['draft1_rc'] = master_port.draft1_r +  master_port.draft2_rc
# master_port['draft0_rc'] = master_port.draft0_r +  master_port.draft1_rc


In [41]:
# Remove selected port 
# list(zip(master_port.LOCODE,master_port.draft0 + master_port.draft1 + master_port.draft2 + master_port.draft3))
master_port['Sum_len_r'] = master_port.draft0_r + master_port.draft1_r + master_port.draft2_r + master_port.draft3_r
master_port['Sum_len_rc'] = master_port.draft0_r + master_port.draft1_r + master_port.draft2_r + master_port.draft3_r
master_port[['LOCODE','Sum_len','Sum_len_r','Sum_len_rc']].sort_values(by='Sum_len',ascending=False).head(14)

,LOCODE,Sum_len,Sum_len_r,Sum_len_rc
58,SGSIN,20239.0,20235,20235
9,CNSHG,16391.0,16130,16130
42,KRPUS,14515.0,14250,14250
6,CNNSA,12939.0,12985,12985
7,CNQDG,10201.0,10145,10145
5,CNNBG,9500.0,9360,9360
15,HKHKG,8980.0,9135,9135
12,CNXMG,8620.0,8590,8590
67,VNSGN,8376.0,8220,8220
16,IDJKT,8335.0,8530,8530


In [22]:
master_port.to_csv('ports_master.csv')
int_country 

In [24]:
# Fill in real througput for topport

cnt = 0
# OPE : original of cargoes, port, economics of port
# PO : port, original of cargoes
for _f,f in enumerate(economics):
    fports = [(_p,p) for c,_p,p in port_country if c == f]
    # print(f,fports,len(fports))
    for _t,t in enumerate(economics):
        if (f==t) or (f in regions_cod and t in regions_cod):
            cnt += 1
            continue
        ## Each port in f
        for _p,p in fports:
            mdl.add_constraint(imp[(_t,_p,f)]<= 
                mdl.sum(port_unload[(p_,o)] for p_,o in po if p == p_ and o!=_f),
                "Import @ {p},{f} for {t} product")         # Import from t to f
            mdl.add_constraint(exp[(_t,_p,f)]<= 
                mdl.sum(port_load[(p_,o)] for p_,o in po if p == p_ and o==_f),
                f"Export @ {p},{f} for {t} product")        # Export from f to t

        if f in int_cod:
        ## Sum every port in f (demand satisfied)
            mdl.add_constraint(mdl.sum(imp[(o,_p,e)] for o,_p,e in ope if o==_t and e == f)
                                == demands[(t,f)],
                                f"Demand (in) satisfy for {f},{t}")
            mdl.add_constraint(mdl.sum(exp[(o,_p,e)] for o,_p,e in ope if o==_f and e == f)
                                == demands[(f,t)],
                                f"Demand (in) satisfy for {f},{t}")
        else:
            ## Sum every port in f (demand relaxed satisfied)
            mdl.add_constraint(mdl.sum(imp[(o,_p,e)] for o,_p,e in ope if o==_t and e == f)
                                <= demands[(t,f)],
                                f"Demand (in) satisfy for {f},{t}")
            mdl.add_constraint(mdl.sum(exp[(o,_p,e)] for o,_p,e in ope if o==_f and e == f)
                                <= demands[(f,t)],
                                f"Demand (in) satisfy for {f},{t}")

In [ ]:
country = pd.read_csv('..//country.csv',na_filter = False).iloc[:,1:]
# country


In [ ]:
cou_reg = pd.Series(country.Economics.values,index=country.LOCODE).dropna().to_dict()
cou_reg[''] = ''
cou_reg['TW'] = 'INT'
econ_zone = dict(zip(country.LOCODE,country.Economics))
len((dict.fromkeys(econ_zone.values())))
# country[['Country_Name','Country_ISO3','Country_Code','Economics']]
econ_zone['TW'] = 'OAS'